<a href="https://colab.research.google.com/github/vonrafael/Self_Attention_Basic/blob/main/convertResizeSmooth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install video2images


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
source = '/content/drive/MyDrive/Vid/16 - The Gates of Judgment.mkv'
des = '/content/drive/MyDrive/Vid/pyscho_frames/'

In [5]:
from video2images import Video2Images


# Let say f = 20 i.e capturing only first 20 frames
Video2Images(video_filepath=source,
             capture_rate=2,
             out_dir=des)

 Video Duration is 0hr: 22min: 51sec 
 The input Video FPS is 23 frames/sec
 Capture rate is 2 frames/sec


 Capturing Frames... :  58%|█████▊    | 19123/32900 [07:13<05:12, 44.14iter/s]


KeyboardInterrupt: ignored

In [6]:
# lets resize them for good


In [7]:
#!/usr/bin/python
from PIL import Image
import os, sys

path = "/content/drive/MyDrive/Vid/pyscho_frames/psycho_pass/"
dirs = os.listdir( path) 

In [10]:
def resize():
    for item in dirs:
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            f, e = os.path.splitext(path+item)
            imResize = im.resize((256,256), Image.LANCZOS)
            imResize.save(f + ' resized.jpg', 'JPEG', quality=90)

In [11]:
resize()

In [12]:
os.chdir(path)

In [13]:
!ls -1|wc -l

3200


In [15]:
!pip install downloader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for downloader: filename=downloader-0.98-py3-none-any.whl size=8134 sha256=6f98654c097b69186b7f641e50687bcaf07a81efda036ffde6a1bd00034b8317
  Stored in directory: /root/.cache/pip/wheels/29/a1/dc/510738d54d4bf7075857e5f42d055fc0f8702c75d6f5fbdaa5
Successfully built downloader


In [17]:
import os
import shutil
import urllib.request

def init_counter(path_to_count_content):
    if not os.path.exists(path_to_count_content):
        return 0
    else:
        already_existing_no_of_files = len(os.listdir(path_to_count_content))
        print("Already downloaded files: ", already_existing_no_of_files)
        return already_existing_no_of_files

def download_non_already_existing_images(filename_with_path, url):
    if not os.path.exists(filename_with_path):
        return download_image(url, filename_with_path)
    else:
        print("Skipping file, already exists: ", filename_with_path)
        return False

def download_image(url, filename):
    try:
        with urllib.request.urlopen(url) as response, open(filename, 'wb') as saving_file:
            shutil.copyfileobj(response, saving_file)
            return True
    except urllib.error.HTTPError as e:
        print(e)
        return False

def zip_images(zipfile_name, path_to_data_to_be_zipped):
    shutil.make_archive(zipfile_name, 'zip', path_to_data_to_be_zipped)

In [18]:
import os
import cv2
from PIL import Image, ImageFilter
# import downloader

PATH_TO_STORED_CARTOON_IMAGES = '/content/drive/MyDrive/Vid/pyscho_frames/psycho_pass/'
PATH_TO_STORE_SMOOTHED_IMAGES = '/content/drive/MyDrive/Vid/pyscho_frames/psychopass_resized/'
SMOOTHED_IMAGES_ZIPFILE_NAME = './psychopass_smoothed'

In [19]:
def edge_smoothing(cartoon_images_filename, smoothed_images_filename):
    print("Edge-smoothing of ", cartoon_images_filename)
    origin = cv2.imread(cartoon_images_filename)
    edges = createEdgesOverlay(origin)
    result = overlayEdges(edges, origin)
    result.save(smoothed_images_filename, "JPEG")

In [20]:
def overlayEdges(edges, origin):
    background = transformFromCV2ToPillowImageFormat(origin)
    background.paste(edges, (0, 0), edges)
    background = background.convert("RGB")
    return background
    

In [21]:
def transformFromCV2ToPillowImageFormat(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGBA)
    return Image.fromarray(img)

In [22]:
def createEdgesOverlay(origin):
    edges = cv2.Canny(origin, 30, 300, 3)
    edges = cv2.dilate(edges, (3, 3))
    edges = cv2.bitwise_not(edges)
    edges = transformFromCV2ToPillowImageFormat(edges)
    makeWhiteBackgroundTransparent(edges)
    edges = edges.filter(ImageFilter.GaussianBlur) #do blurring here because doing it before making background transparent results in white halo

    return edges

In [23]:
def makeWhiteBackgroundTransparent(img):
    datas = img.getdata()
    newData = []
    for item in datas:
        if item[0] == 255 and item[1] == 255 and item[2] == 255:
            newData.append((255, 255, 255, 0))
        else:
            newData.append(item)
    img.putdata(newData)

In [24]:
def main():
    if not os.path.exists(PATH_TO_STORED_CARTOON_IMAGES):
        print("Can not smooth images, path does not exist: ", PATH_TO_STORED_CARTOON_IMAGES)

    if not os.path.exists(PATH_TO_STORE_SMOOTHED_IMAGES):
        os.makedirs(PATH_TO_STORE_SMOOTHED_IMAGES)

    for filename in os.listdir(PATH_TO_STORED_CARTOON_IMAGES):
        cartoon_images_filename = PATH_TO_STORED_CARTOON_IMAGES + filename
        smoothed_images_filename = PATH_TO_STORE_SMOOTHED_IMAGES + filename

        if not os.path.exists(smoothed_images_filename):
            edge_smoothing(cartoon_images_filename, smoothed_images_filename)
        else:
            print("Skipping file, already exists, ", cartoon_images_filename)

    downloader.zip_images(SMOOTHED_IMAGES_ZIPFILE_NAME, PATH_TO_STORE_SMOOTHED_IMAGES)

In [ ]:
main()

In [26]:
os.chdir('/content/drive/MyDrive/Vid/pyscho_frames/psychopass_resized/')

In [28]:
!ls -1|wc -l

3200
